<a href="https://colab.research.google.com/github/ihightower/my-first-binder/blob/main/pe084.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [77]:
# !pip install pandas

In [78]:
print('last saved 11nov 14:05')

last saved 11nov 14:05


In [79]:
import random
import pandas as pd
import time


In [80]:
# del logging, logger, ch, formatter
from importlib import reload
import logging
logging.shutdown()
reload(logging)


# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

 # create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
#ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)


# Project Euler P084

<img src="https://projecteuler.net/project/images/p084_monopoly_board.png">




In the game, Monopoly, the standard board is set up in the following way:

Community Chest (2/16 cards):
* Advance to GO
* Go to JAIL

Chance (10/16 cards):

* Advance to GO
* Go to JAIL
* Go to C1
* Go to E3
* Go to H2
* Go to R1
* Go to next R (railway company)
* Go to next R
* Go to next U (utility company)
* Go back 3 squares.


The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.

By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.

Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.

If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.

# Code Starts

## Set Monopoly Squares

In [81]:
#pe084 - Monopoly odds
lst_monopoly_squares = [
    'GO', 'A1', 'CC1', 'A2', 'T1', 'R1', 'B1', 'CH1', 'B2', 'B3', 'JAIL', 'C1',
    'U1', 'C2', 'C3', 'R2', 'D1', 'CC2', 'D2', 'D3', 'FP', 'E1', 'CH2', 'E2',
    'E3', 'R3', 'F1', 'F2', 'U2', 'F3', 'G2J', 'G1', 'G2', 'CC3', 'G3', 'R4',
    'CH3', 'H1', 'T2', 'H2'
]

lst_comchest = [0, 10] + [99] * 14
random.shuffle(lst_comchest)

lst_chance = [0, 10, 11, 24, 39, 5, [5, 15, 25, 35], [5, 15, 25, 35], [12, 28], -3] + [99] * 6
random.shuffle(lst_chance)

print(lst_comchest)
print(lst_chance)



[99, 99, 99, 99, 99, 99, 99, 10, 99, 99, 99, 99, 0, 99, 99, 99]
[10, 24, [5, 15, 25, 35], 99, 11, 39, [12, 28], 99, 99, 99, -3, 5, [5, 15, 25, 35], 99, 0, 99]


In [82]:
lst_comchest_pos = [2, 17, 33]
lst_chance_pos = [7, 22, 36]

In [83]:
print('comechest cards: ', lst_comchest)
print('chance cards: ',lst_chance)

comechest cards:  [99, 99, 99, 99, 99, 99, 99, 10, 99, 99, 99, 99, 0, 99, 99, 99]
chance cards:  [10, 24, [5, 15, 25, 35], 99, 11, 39, [12, 28], 99, 99, 99, -3, 5, [5, 15, 25, 35], 99, 0, 99]


## Functions

### func: move_from

In [131]:
def move_from(currpos):
    
    global gRollingDoubles
    
    dice_sides = 6
    
    dice1 = random.randint(1, dice_sides)
    dice2 = random.randint(1, dice_sides)
    
    dice_roll = dice1 + dice2
    
    if dice1 == dice2:
        gRollingDoubles += 1
    else:
        gRollingDoubles = 0
 
    if gRollingDoubles == 1:
        logger.debug(f'       rolled 1st double in {currpos}')
        # logger.setLevel(logging.WARNING)
        pass
    
    if gRollingDoubles == 2:
        logger.warning(f'       rolled 2nd double in {currpos}')
        # logger.setLevel(logging.WARNING)
        pass
    
    if gRollingDoubles == 3:
        # logger.setLevel(logging.DEBUG)
        
        logger.critical(f'       rolled 3rd double in {currpos}... and now going to jail')
 #         print('rolled 3 doubles, going to jail.. from ', newpos)
        newpos = 10 # go to jail
        gRollingDoubles = 0
        
        logger.critical(f'...... dice rolled: {dice1}+{dice2}={dice_roll}, from: {currpos} to: {newpos}')
        return newpos

    
    newpos = (currpos + dice_roll) % 40
 
 
    logger.debug(f'...... dice rolled: {dice1}+{dice2}={dice_roll}, from: {currpos} to: {newpos}')
    
    
    if newpos == 20:
        logger.debug(f'in {newpos}... should go to jail. RollingDouble#:{gRollingDoubles}')
        newpos = 10
        gRollingDoubles = 0


    

 #     print('\n...... dice rolled', currpos, dice_roll, newpos)
    return newpos



```
...... dice rolled: 6, from: 25 to: 31
...... dice rolled: 9, from: 31 to: 0
rolled 1st double in 0
...... dice rolled: 10, from: 0 to: 10
...... dice rolled: 10, from: 10 to: 10
```



### func: take_chance_card

In [85]:
# chace cards:  [11, -3, [12, 28], 99, 99, [5, 15, 25, 35], 99, 0, [5, 15, 25, 35], 39, 99, 99, 24, 5, 10, 99]
def take_chance_card(currpos):
    global gCHANCELOC
    newpos = currpos
    
    chance_card = lst_chance[gCHANCELOC]
    logger.info(f'  in pos:{currpos}, seq: {gCHANCELOC}, card: {chance_card} {type(chance_card)}')
 #     print('  in pos:', currpos, ' seq: ', gCHANCELOC, ' card: ',  chance_card, type(chance_card))

    if (chance_card == 99):
        newpos = currpos
        pass
 #         print(' STAYING PUT RIGHT HERE')
    elif (type(chance_card) is int and chance_card < 0):
        newpos = currpos + chance_card
        newpos %= 40
        
 #         print('>>> moving to: ', newpos)
        
    elif (type(chance_card) is int):   
        newpos = chance_card
 #         print('>>> moving to: ', newpos)
        
    else:
 #         print('>>> moving to: ', 'sorry i am a list, i think')
                
        if currpos > chance_card[-1]:
            newpos = chance_card[0]
        else:    
            for i in chance_card:
                if currpos > i:
                    continue
                else:
                    newpos = i
                    break
        
    
    
    gCHANCELOC += 1
    gCHANCELOC %= len(lst_chance)
    
    return newpos
    

### func: take_comchest_card

In [86]:
# comechest cards:  [99, 99, 99, 0, 99, 99, 99, 99, 99, 10, 99, 99, 99, 99, 99, 99]

def take_comchest_card(currpos):
    global gCOMCHESTLOC
    newpos = currpos
    
    comchest_card = lst_comchest[gCOMCHESTLOC]
    logger.info(f'  in pos:{currpos}, seq: {gCOMCHESTLOC}, card: {comchest_card} {type(comchest_card)}')
  #     print('  in pos:', currpos, ' seq: ', gCOMCHESTLOC, ' card: ', comchest_card, type(comchest_card))
    
    if (comchest_card == 99):
        newpos = currpos
        pass
 #         print(' STAYING PUT RIGHT HERE')
    elif (type(comchest_card) is int):
        newpos = comchest_card
 #         print('>>> moving to: ', newpos)
        
    else:
        pass
 #         print('>>> moving to: ', 'sorry i am a list, i think')

        
        
        if currpos > comchest_card[-1]:
            newpos = comchest_card[0]
        else:    
            for i in comchest_card:
                if currpos > i:
                    continue
                else:
                    newpos = i
                    break        
        
    gCOMCHESTLOC += 1
    gCOMCHESTLOC %= len(lst_comchest)
    
    return newpos

### func: run_simulation

In [124]:
def run_simulation(simcount, currpos=0):
  for simu in range(simcount):
      # time.sleep(1)

      currpos = move_from(currpos)
      
  #     print(' in position now:', currpos, lst_monopoly_squares[currpos])
      
      if currpos in lst_chance_pos:
          
          # logger.setLevel(logging.CRITICAL)
          
          logger.info('\n>>> i am taking chance')
  #         print('\n>>> i am taking chance')
          currpos = take_chance_card(currpos)
          logger.info(f'   moved to {currpos}\n')
          
          # logger.setLevel(logging.INFO)
  #         print('.... moved to', currpos)
          pass
          #print(mychance_card)
      
      if currpos in lst_comchest_pos:
          # logger.setLevel(logging.CRITICAL)
          # logger.setLevel(logging.INFO)
          logger.info('\n>>> i am taking community chest')
  #         print('\n>>> i am community chest')
          currpos = take_comchest_card(currpos)
          logger.info(f'   moved to {currpos}\n')
          # logger.setLevel(logging.INFO)

  #         print('.... moved to', currpos)
          #print(mycomchest_card)
          
      if gRollingDoubles == 0:
        dictsq[currpos] = dictsq.get(currpos, 0) + 1

  #     print(dice2, cuminsquare, insquare)

  print(dict(sorted(dictsq.items())))

  data_items = dictsq.items()
  data_list = list(data_items)

  df = pd.DataFrame(data_list)
  return df
  #df.sort_values([1], ascending=False)

 

# Run Simulation

<a href="https://projecteuler.net/problem=84">problem link</a>

In [117]:
df.sum()

0        760
1    1000000
dtype: int64

In [132]:
 # answer: 102400

currpos = 0 #helllo
dictsq = {}

gCOMCHESTLOC = 0
gCHANCELOC = 0
gRollingDoubles = 0

# logger.setLevel(logging.WARNING)
logger.setLevel(logging.CRITICAL)

simcount = 10000

df = run_simulation(simcount)
df['pct'] = df.iloc[:,1]/simcount *100

df.sort_values([1], ascending=False)

       rolled 3rd double in 3... and now going to jail
...... dice rolled: 4+4=8, from: 3 to: 10
       rolled 3rd double in 14... and now going to jail
...... dice rolled: 4+4=8, from: 14 to: 10
       rolled 3rd double in 15... and now going to jail
...... dice rolled: 1+1=2, from: 15 to: 10
       rolled 3rd double in 0... and now going to jail
...... dice rolled: 5+5=10, from: 0 to: 10
       rolled 3rd double in 4... and now going to jail
...... dice rolled: 6+6=12, from: 4 to: 10
       rolled 3rd double in 31... and now going to jail
...... dice rolled: 6+6=12, from: 31 to: 10
       rolled 3rd double in 15... and now going to jail
...... dice rolled: 5+5=10, from: 15 to: 10
       rolled 3rd double in 4... and now going to jail
...... dice rolled: 6+6=12, from: 4 to: 10
       rolled 3rd double in 39... and now going to jail
...... dice rolled: 4+4=8, from: 39 to: 10
       rolled 3rd double in 6... and now going to jail
...... dice rolled: 3+3=6, from: 6 to: 10
       rolled 3

{0: 279, 1: 174, 2: 168, 3: 193, 4: 218, 5: 249, 6: 177, 7: 79, 8: 190, 9: 194, 10: 593, 11: 227, 12: 230, 13: 218, 14: 194, 15: 257, 16: 221, 17: 253, 18: 273, 19: 280, 21: 251, 22: 82, 23: 228, 24: 277, 25: 215, 26: 241, 27: 183, 28: 219, 29: 187, 30: 184, 31: 198, 32: 213, 33: 201, 34: 203, 35: 231, 36: 74, 37: 192, 38: 192, 39: 222}


,0,1,pct
0,10,593,5.93
31,19,280,2.80
30,0,279,2.79
4,24,277,2.77
12,18,273,2.73
1,15,257,2.57
25,17,253,2.53
21,21,251,2.51
6,5,249,2.49
33,26,241,2.41


,0,1
3,10,68
10,5,41
0,11,36
11,15,34
6,28,33
29,16,32
1,19,31
12,18,31
30,21,30
19,37,30


# Testing Area

In [90]:
1/0

ZeroDivisionError: ignored

<a id="section-one"></a>
## Helper Functions

**^html links hidden here, fork notebook then click this cell to see it**
<a id="section-two"></a>
## Section Two
**^html links hidden here, fork notebook then click this cell to see it**


In [ ]:
import re

lst1 = ['JAIL', 'C1', 'E3', 'H2', 'R1']

for x in lst1:    
    print(x, lst_monopoly_squares.index(x))

lst2 = ['^R', '^U', '^CC', '^CH']

for x in lst2:    
    r = re.compile(x)
    new_list = list(filter(r.match, lst_monopoly_squares))
    print(new_list)
    
    for y in new_list:
        print(x, lst_monopoly_squares.index(y))

## Image Testing (Works!)

In [ ]:
# Importing the PIL library
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
import requests
from io import BytesIO
url = "https://projecteuler.net/project/images/p084_monopoly_board.png"
response = requests.get(url)
orig_img = Image.open(BytesIO(response.content))

In [ ]:
# Open an Image
img = orig_img.copy()

myfont = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 20)

# myfont = ImageFont.truetype("arial.ttf", 1)
 
# Call draw Method to add 2D graphics in an image
I1 = ImageDraw.Draw(img) 

mysq = 0

for i in range(4):
  
  
  if i == 0:
    print ('walk right 0 to 9')
    for sq in range(10):
      x = 40.7*sq + 10
      y = 37.8*i + 5

      m = lst_monopoly_squares[mysq]
      mysq+=1

      
      I1.text((x, y),f'{m}', font=myfont, fill =(255, 0, 0))

  if i == 1:
    print ('walk down 0 to 9')
   
    for sq in range(10):
      x = 40.7*10 + 10
      y = 37.8*sq + 5
      m = lst_monopoly_squares[mysq]
      mysq+=1      
      I1.text((x, y),f'{m}',font=myfont, fill =(255, 0, 0))

  if i == 2:
    print ('walk left 0 to 9')

    for sq in range(10, 0, -1):
      x = 40.7*sq + 10
      y = 37.8*10 #+ 5
      m = lst_monopoly_squares[mysq]
      mysq+=1      
      I1.text((x, y),f'{m}', font=myfont,fill =(255, 0, 0))
  
  if i == 3:
    print ('walk up 0 to 9')

    for sq in range(10, 0, -1):
      x = 40.7*0 + 10
      y = 37.8*sq + 3
      m = lst_monopoly_squares[mysq]
      mysq+=1      
      I1.text((x, y),f'{m}', font=myfont,fill =(255, 0, 0))

 # Display edited image
img

In [ ]:
# Open an Image
img = orig_img.copy()
 
# Call draw Method to add 2D graphics in an image
I1 = ImageDraw.Draw(img)

for sq in range(40):
  
  x = 40.7*i + 10
  for j in range(11):
    if (i >=1 and i <= 9) and (j>=1 and j<=9):
      continue
    
  
    y = 37.8*j + 5

    I1.text((x, y), f'{i},{j}', fill =(255, 0, 0))
  
# Display edited image
img


In [ ]:
# Open an Image
img = orig_img.copy()
 
# Call draw Method to add 2D graphics in an image
I1 = ImageDraw.Draw(img)
 
# Custom font style and font size
# myFont = ImageFont.truetype('FreeMono.ttf', 65)
 
# Add Text to an image

for i in range(11):
  x = 40.7*i + 10
  for j in range(11):
    if (i >=1 and i <= 9) and (j>=1 and j<=9):
      continue
    
  
    y = 37.8*j + 5

    I1.text((x, y), f'{i},{j}', fill =(255, 0, 0))
  
# Display edited image
img


## Simple Test1

In [ ]:
mylist = [5, 15, 25, 35]

currno = 36

if currno > mylist[-1]:
    selection = mylist[0]
else:    
    for i in mylist:
        if currno > i:
            continue
        else:
            selection = i
            break

print(selection)
    

## Test Markdown

Clicking on each link jumps you to the section that is marked. It is really intuitive to use! 
I would recommend adding a table of contents to any of your published notebooks, so that the reader doesn't get lost.
You would want everyone to enjoy reading that kernel that you spent so long on!

Links use HTML to target your browser to that specific section of your Kaggle notebook.
To see the inner workings and code, please fork this notebook and click on the cells to see the markdown code

* [Introduction](#section-one)
* [Body](#section-two)
    - [Subsectionaaa 1](#subsection-one)
    - [Subsectionaaa 2](#anything-you-like)
* [Conclusionaaa](#section-three)

## Logging Test

In [ ]:
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.info('warn message')
logger.error('error message')
logger.critical('critical message')